# Basic example
This is a basic notebook, used only to detect and track objects

In [1]:
import sys
#! still dont fully understand sys.path.insert usage; navigating the directories?
sys.path.insert(0,'../..')
sys.path.insert(0,'..')

import cv2
import os
import pickle 

#! Allows notebook to only use GPU 0?
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

from random import randint
import matplotlib.pyplot as plt

from pathlib import Path

from time import time

# Deep sort imports
from deep_sort import nn_matching

import config

%matplotlib inline 


In [2]:
from cav.detection import ObjectDetector

# Here should be a patch to your object detection model. 
#! patch or path? Also no models folder as is - use pretrained?
od = ObjectDetector(config.MODEL_PATH)

#### Parameters

In [3]:
### Setting up parameters for object detection loop
#! unclear what CUT_UPPER is; move below?
#! maybe to crop out the sky?
CUT_UPPER = 0
# Makes path to save detections and creates the folder
SAVE_DETECTIONS = os.path.join(config.DATA_PATH, 'detections.p')
Path(os.sep.join(SAVE_DETECTIONS.split(os.sep)[:-1])).mkdir(parents=True, exist_ok=True)

# Reads example video saved from part 01
VIDEO_FILE = pickle.load(open(f'{config.DATA_PATH}/videopath.p', 'rb'))

In [4]:
### Runs object detection on example video
#! loads video?
cap = cv2.VideoCapture(VIDEO_FILE)

#! objects, results, colors not used
objects = []
results = []
colors = {}

save_detections = {}    

i = 0
t = time()

while cap.isOpened():
    # get width, height, and fps of video
    video_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
    video_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - CUT_UPPER     
    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_timeStamp = i/fps

    #! time since beginning of while loop?
    #! if so can rename better
    t2 = time() - t
    sys.stdout.write('{} frames done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
        i, t2, i/t2))                   
    i += 1
    
    # gets next frame
    #! ret not used
    ret, image = cap.read()
    # no more frames left
    if image is None:
        break

    #! are the 3 dimensions not RGB?
    if CUT_UPPER > 0:
        image = image[CUT_UPPER:, :, :]
    
    #! trying to detect move over event? 
    #! what is it drawing bounding boxes around?
    boxes, scores, classes = od.detect(image, timestamp=frame_timeStamp)
    if SAVE_DETECTIONS is not None:
        save_detections[i] = (boxes, scores, classes)
    
cap.release()

if SAVE_DETECTIONS is not None:
    pickle.dump(save_detections, open(SAVE_DETECTIONS,'wb'))
    print (f'\nDetections saved in {SAVE_DETECTIONS}.')



903 frames done in 3692.3 seconds (0.24 frames/sec)    
Detections saved in ./data/detections.p.


In [5]:
#! used within the loop?
print ('({}x{}), {:.1f} fps'.format(video_x, video_y, fps))

(1920x1080), 30.0 fps
